1️⃣ RAG with the new LangChain API 


✅ Stack

langchain

langchain-ollama

faiss

Ollama (llama3, nomic-embed-text)

##### StreamLit user interface

First I had to install ipykernal through following command:
```python
conda install -n rag-chatbot ipykernel --update-deps --force-reinstall
```


| Use case               | Best choice           |
| ---------------------- | --------------------- |
| Just learning / hobby  | Sentence-Transformers |
| RAG app, local dev     | Ollama                |
| Production SaaS        | Paid OpenAI key       |
| Offline / private data | Local models          |


In [ ]:
## This is for .env file if we use openAI API keys..
%load_ext dotenv
%dotenv

##### Download and Install Ollama from https://ollama.com/

A. Install packages

In [ ]:
!pip install -U langchain langchain-ollama langchain-community faiss-cpu

The ollama pip package ≠ Ollama itself
It’s just a Python client that talks to the Ollama server.

B. Load and split documents

In [4]:
from langchain_community.document_loaders import PyPDFLoader,TextLoader
from langchain_text_splitters import TokenTextSplitter, RecursiveCharacterTextSplitter
from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.messages import SystemMessage
from langchain_core.prompts import (PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate)
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import (ChatOpenAI, OpenAIEmbeddings)
from langchain_chroma import Chroma



Loader_pdf = PyPDFLoader('1._Intro_to_AI_-_Course_notes.pdf')
#Loader_pdf = TextLoader('Alan_Turing.txt')

doc_list   = Loader_pdf.load()

### Clean pdf text
def clean_pdf_text(text):
    return (
        text.replace("\x00", "")   # null bytes
            .replace("\n", " ")    # line breaks
            .strip()
    )

for doc in doc_list:
    doc.page_content = clean_pdf_text(doc.page_content)




from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

chunks = splitter.split_documents(doc_list)

[t.page_content for t in chunks if "Turing" in t.page_content][:2]




#embedding = OpenAIEmbeddings(model='text-embedding-3-small')
#vectorstore = Chroma.from_documents(documents=doc_list_token_split, embedding=embedding, persist_directory="intro-to-ai")



['of AI     The evolution of Artificial Intelligence: Key milestones  Early Beginnings  • 1950 - Alan Turing\'s Seminal Question: Alan Turing publishes a paper  asking, "Can machines think?" and introduces the Turing Test, setting a  practical criterion for evaluating machine intelligence. If an interrogator  can\'t distinguish between responses from a machine and a human, the  machine is deemed to exhibit human-like intelligence.  Formal Recognition  • 1956 - Dartmouth Conference: The term',
 "ChatGPT can perform a broad range of tasks:  o Writing jokes  o Proofreading texts  o Recommending actions  o Creating visuals and formulas  o Solving mathematical problems  • Relation to Turing Test: ChatGPT's ability to generate human-like responses  aligns with Alan Turing's imitation game concept, suggesting it can pass the  Turing Test, thus classifying it as semi-strong AI.    Artificial General Intelligence (AGI)  • Definition and Goal: AGI, also known as strong AI, aims to create"]

In [ ]:
#!pip install -U langchain-ollama
!ollama pull llama3 
!ollama pull nomic-embed-text
#!ollama run llama3

C. Embeddings

In [6]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="nomic-embed-text"
)

D. Vector store

In [7]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(
    chunks,
    embeddings
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 4})


docs = retriever.invoke("What did Alan Turing do?")

for d in docs:
    print("----")
    print(d.page_content[:300])


----
RAG (Retrieval-Augmented  Generation), and fine-tuning.  • Demand for AI Expertise: As AI continues to be a central element of  business strategy, there is a growing need for skilled AI engineers who can  navigate these tools to enhance business applications and outputs.
----
365 DATA SCENCE                                                                                                                            2            Table of Contents   ................................................................................................................................
----
3  2. Data is essential for building AI ....................................................................................................... 7  3. Key AI techniques .......................................................................................................................... 10  4. Im
----
365 DATA SCENCE                                                                                           

E. LLM (new API)

In [8]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(
    model="llama3",
    temperature=0
)


F. Prompt

In [9]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
You are a helpful assistant.
Use the context below to answer the question.

Context:
{context}

Question:
{question}
""")

G. RAG chain (modern langchain)

In [10]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
)

H. Invoke

In [ ]:
print(rag_chain.invoke("What did Alan Turing do?"))
